# Types of Chaining

## 1. Extendend or sequential chaining
chaining tasks one by one in a sequential line

## 2. parallel chaining
lets you run tasks without depending on other tasks 

## 3. conditional chaining
lets you run a particular branch based on a condition


In [3]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

def main():
    # Initialize the language model
    llm = ChatOpenAI(model="gpt-4", temperature=0.0)

    # Define the prompt template
    # prompt = PromptTemplate(
    #     input_variables=["question"],
    #     template="Answer the question: {question}",
    # )

    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant in the {field}."),
        ("human", "{question}"),
    ])

    # Create the output parser
    output_parser = StrOutputParser()

    # Combine the components into a single chain
    chain = prompt | llm | output_parser

    # Run the chain with a sample question
    result = chain.invoke({"field": "computer science", "question": "What is the Turing test?"})
    
    print(result)
if __name__ == "__main__":
    main()
# This script uses LangChain to create a simple question-answering chain with OpenAI's GPT-4 model.

The Turing Test is a method of inquiry in artificial intelligence (AI) for determining whether or not a computer is capable of human-like intelligence. Proposed by the British mathematician and computer scientist Alan Turing in 1950, the test involves a human evaluator who carries out natural language conversations with another human and a machine designed to generate human-like responses. The evaluator knows that one of the two partners in conversation is a machine, and all participants are separated from one another. If the evaluator cannot reliably tell the machine from the human, the machine is said to have passed the test. The test does not check the ability to give correct answers, it checks how closely the answer resembles typical human answers.


# RUNNABLES:- To provide extra processing, this are the inner workings of the pipe operator


In [8]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema.runnable import RunnableLambda, RunnableSequence
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()
def main(): 
    # Initialize the language model
    llm = ChatOpenAI(model="gpt-4", temperature=0.0)

    # Define the prompt template
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant in the {field}."),
        ("human", "{question}"),
    ])

    # Create a simple chain using RunnableSequence
    chain = RunnableSequence(
        prompt,
        llm,
        RunnableLambda(lambda x: x.content)
    )

    # Run the chain with a sample question
    result = chain.invoke({"field": "computer science", "question": "What is the Turing test?"})
    
    print(result)
if __name__ == "__main__":
    main()
# This script uses LangChain to create a simple question-answering chain with OpenAI's GPT-4 model.


The Turing Test is a method of inquiry in artificial intelligence (AI) for determining whether or not a computer is capable of human-like intelligence. It was proposed by the British mathematician and computer scientist Alan Turing in 1950.

In the test, a human evaluator interacts with a computer program and a human through a computer interface. If the evaluator cannot reliably tell the program from the human, the machine is said to have passed the test and demonstrated human-like intelligence.

However, it's important to note that passing the Turing Test doesn't necessarily mean the machine has the same consciousness or understanding as a human. It simply means it was able to produce responses that were indistinguishable from those of a human in that specific context.


# Sequential Chaining

In [15]:
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnableSequence
load_dotenv()   

def main():
    # Initialize the language model
    llm = ChatOpenAI(model="gpt-4", temperature=0.0)

    # Define the prompt template
    prompt_facts = ChatPromptTemplate.from_messages([
        ("system", "You like telling facts about {field}."),
        ("human", "Tell me {count} facts."),
    ])

    prompt_traslations = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful translator."),
        ("human", "Translate '{text}' to {language}."),
    ])

    count_words = RunnableLambda(lambda x: len(x.split()))
    prepare_for_translation = RunnableLambda(lambda x: {
        "text": x,
        "language": "Somali"
    })

    # Create a chain for facts
    # facts_chain = RunnableSequence(
    #     prompt_facts,
    #     llm,
    #     StrOutputParser()
    # )
    # # Create a chain for translations
    # chain = RunnableSequence(
    #     facts_chain,
    #     count_words,
    #     prepare_for_translation,
    #     prompt_traslations,
    #     llm,
    #     StrOutputParser()
    # )

    chain = prompt_facts | llm | StrOutputParser() | prepare_for_translation | prompt_traslations | llm | StrOutputParser()


    # Run the chain with a sample question
    result = chain.invoke({"field": "computer science", "count": "3"})
    
    print(result)

if __name__ == "__main__":
    main()

1. Kombiyuutarka elektroniga ee ugu horeeyay ee ENIAC wuxuu miisaaniyadda ka badnaa 27 oo tan, wuxuuna ka dhacayaa 1800 fut koonfur. Waxaa soo saaray Ciidamada Mareykanka ee Dagaalkii Adduunka ee Labaad, waxaana loogu adeegsaday in lagu xisaabiyo miisaska dagaalka.

2. Qofka ugu horeysay ee barnaamijka kombiyuutarka waxay ahayd haweeney lagu magacaabo Ada Lovelace. Waxay ka shaqaysay kombiyuutarka guud ee hore ee farsamo ee Charles Babbage, oo loo yaqaanay Injineerka Farsamo, waxayna qortay xeerka mashiinka ee ugu horeeya ee adduunka ee loogu talagalay mashiin farsamo oo kaliya ku jirta warqad.

3. Fayraska kombiyuutarka ugu horeeyay waxaa sameeyay 1971-kii Robert Thomas, injineer ka tirsan shirkadda BBN Technologies. Lagu magacaabay "Creeper", wuxuu ahaa barnaamij is-dupliki ah oo tijaabo ah oo aan ahayn mid waxyeelo leh, laakiin waxaa loo sameeyay in lagu muujiyo barnaamijka gacanta.


# Parallel Chaining: Write a blog post to analyze the movie inception. Chain 1 analyzes the plot chain 2 analyzes the charater


In [26]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnableParallel
from dotenv import load_dotenv
load_dotenv()

def analyze_movie(plot):
    plot_template = ChatPromptTemplate.from_messages([
        ("system", "You are a movie critic."),
        ("human", "analyze the plot{plot}. what are the strong and weak points?"),
    ])
    return plot_template.format_prompt(plot=plot)
    

def analyze_characters(characters):
    character_template = ChatPromptTemplate.from_messages([
        ("system", "You are a movie critic."),
        ("human", "analyze the characters in the movie {character}. what are their strengths and weaknesses?"),
    ])
    return character_template.format_prompt(character=characters)

def combine_verdicts(plot_verdict, character_verdict):
    return f"Plot Verdict: {plot_verdict}\nCharacter Verdict: {character_verdict}"

def main():
    # Initialize the language model
    llm = ChatOpenAI(model="gpt-4", temperature=0.0)

    # Define the prompt template
    summarizer = ChatPromptTemplate.from_messages([
        ("system", "You are a movie critic "),
        ("human", "provide a brief summary on the movie {movie_name}"),
    ])

    # Create a chain for analyzing the movie plot
    plot_chain = RunnableLambda(lambda x: analyze_movie(x)) | llm | StrOutputParser()
    # Create a chain for analyzing the characters
    character_chain = RunnableLambda(lambda x: analyze_characters(x)) | llm | StrOutputParser()
    # Combine the two chains in parallel
    parallel_chain = RunnableParallel(
        branches={
            "plot": plot_chain,
            "character": character_chain
        },
    )
    # Combine the verdicts
    combine_chain = RunnableLambda(lambda x: combine_verdicts(x["branches"]["plot"], x["branches"]["character"]))
    # Create the final chain
    # final_chain = RunnableSequence(
    #     summarizer,
    #     llm,
    #     StrOutputParser(),
    #     parallel_chain,
    #     # combine_chain
    # )
    final_chain = summarizer | llm | StrOutputParser() | parallel_chain | combine_chain
    # Run the chain with a sample movie
    result = final_chain.invoke({"movie_name": "Inception"})
    print(result)

if __name__ == "__main__":
    main()


    

Plot Verdict: "Inception" is a masterclass in storytelling, with its strongest points being its intricate plot, compelling characters, and innovative exploration of the subconscious mind. The film's complex narrative structure, which involves multiple layers of dreams within dreams, is a testament to Christopher Nolan's genius as a writer and director. The plot is not only intellectually stimulating but also emotionally engaging, thanks to the deeply personal stakes for the protagonist, Dom Cobb. The film's exploration of themes such as reality, illusion, and the personal cost of survival adds depth and philosophical weight to the story.

The performances, particularly by Leonardo DiCaprio, are another strong point. DiCaprio brings a sense of vulnerability and desperation to his character, making the audience empathize with his plight. The supporting cast, including Ellen Page, Joseph Gordon-Levitt, and Tom Hardy, also deliver strong performances.

The film's visual effects and action 

# Conditional Chaining
project example user feedback analysis

In [36]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnableSequence, RunnableBranch
from dotenv import load_dotenv
load_dotenv()

def main():
    # Initialize the language model
    llm = ChatOpenAI(model="gpt-4", temperature=0.0)

    # Define the prompt template
    classification = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant in customer feedback."),
        ("human", "Classify the feedback: {feedback} into positive, negative, or neutral."),
    ])
    classification_chain = classification | llm | StrOutputParser()
    positive_feedback = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant in customer feedback."),
        ("human", "Provide a positive response to the feedback: {feedback}"),
    ])
    negative_feedback = ChatPromptTemplate.from_messages([  
        ("system", "You are a helpful assistant in customer feedback."),
        ("human", "Provide a negative response to the feedback: {feedback}"),
    ])
    neutral_feedback = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant in customer feedback."),
        ("human", "Provide a neutral response to the feedback: {feedback}"),
    ])
    escalate_feedback = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant in customer feedback."),
        ("human", "Escalate the feedback: {feedback} to the appropriate team."),
    ])
    branches = RunnableBranch(
        
            (lambda x: "positive" in x, positive_feedback| llm | StrOutputParser()),
            (lambda x:"negative" in x, negative_feedback | llm | StrOutputParser()),
            (lambda x: "neutral" in x, neutral_feedback | llm | StrOutputParser()),
        
            escalate_feedback | llm | StrOutputParser()
    )
    chain = RunnableSequence(
        classification_chain,
        branches
    )
    result = chain.invoke({"feedback": "Terrible service, I will never come back!"})
    print(f"Classification Result: {result}")

if __name__ == "__main__":
    main()
    




Classification Result: Dear Customer,

We're sorry to hear about your negative experience. We always aim to deliver a great experience, and we are gutted when we don’t meet expectations. Thanks for taking the time to bring this to our attention. We will use the feedback to make us better and to ensure this doesn’t happen again. 

Best Regards,
[Your Name]
